## Minimize and equilibrate
 - Minimize
 - Equilibrate
 
 Sanity checks along the way.
 
 Protocol files are copied from https://github.com/intbio/gmx_protocols/tree/master/amber into
Protocols folder.
If others are needed modify then in the folder.

Theoretically this notebook could be exectuted in a single step, but it is unclear currently how stable the execution of the notebook is if you close browser window or connection in interrupted.

An alternative is to run from a persistent (opened via X2Go or using Tmux) terminal as

```
jupyter nbconvert --ExecutePreprocessor.timeout=72000 --ExecutePreprocessor.kernel_name=python3 --to notebook --execute 2_min_equil.ipynb --output 2_min_equil.ipynb
```



In [78]:
#Libraries and main variables initialized
import MDAnalysis as mda
from MDAnalysis.analysis import align
from MDAnalysis.analysis.rms import rmsd
import numpy as np
import urllib.request, json
import os
import re
import nglview as nv
from funcs import get_files_from_git
from funcs import view_nucl

#Reloading funcs if changed
from importlib import reload 
import funcs,config
reload(funcs)
reload(config)



%matplotlib inline

from IPython.display import Markdown, display
from config import folder,descr,pname,ssh_host
display(Markdown(descr))
print("Folder set to: ",folder)
print("Project name: ",pname)
print("SSH host set to:",ssh_host)




# MD simulations of 1kx5 nucleosome core particle, original full tails symmetrized
- AMBER14SB force field
- 150 mM NaCl
- box 2nm
- Since tail conformations on both sides are vastly different and the box would be too big. We used chimera to substitute histone copies with their symmetry mates. The copies used as reference are chains A B C D. 1kx5_sym.pdb is the result. Crystall water was not retained.


Folder set to:  1kx5_sym
Project name:  1kx5_sym
SSH host set to: lomo2


### 1_Minimization - steepest descent gradient method 10000 steps with POSRED constraints

#### Prepare tpr files using GROMPP
- http://manual.gromacs.org/documentation/current/onlinehelp/gmx-grompp.html

In [79]:
%%bash
#Prepare system
rm -rf GMX_run
mkdir -p GMX_run
gmx grompp -f MDProtocols/1_minim.mdp -c GMX_system/init_solv_ions.pdb -p GMX_system/topol.top \
-o GMX_run/1_minim.tpr -r GMX_system/init_solv_ions.pdb -n GMX_system/index.ndx -po GMX_run/mdout_1_minim.mdp


Calculating fourier grid dimensions for X Y Z
Using a fourier grid of 160x160x160, spacing 0.116 0.116 0.116
This run will generate roughly 183 Mb of data


gmx: Relink `/opt/miniconda3/envs/moldyn/bin/../lib/././libgfortran.so.4' with `/lib/x86_64-linux-gnu/librt.so.1' for IFUNC symbol `clock_gettime'
                      :-) GROMACS - gmx grompp, 2018.4 (-:

                            GROMACS is written by:
     Emile Apol      Rossen Apostolov      Paul Bauer     Herman J.C. Berendsen
    Par Bjelkmar    Aldert van Buuren   Rudi van Drunen     Anton Feenstra  
  Gerrit Groenhof    Aleksei Iupinov   Christoph Junghans   Anca Hamuraru   
 Vincent Hindriksen Dimitrios Karkoulis    Peter Kasson        Jiri Kraus    
  Carsten Kutzner      Per Larsson      Justin A. Lemkul    Viveca Lindahl  
  Magnus Lundborg   Pieter Meulenhoff    Erik Marklund      Teemu Murtola   
    Szilard Pall       Sander Pronk      Roland Schulz     Alexey Shvetsov  
   Michael Shirts     Alfons Sijbers     Peter Tieleman    Teemu Virolainen 
 Christian Wennberg    Maarten Wolf   
                           and the project leaders:
        Mark Abraham, Berk Hess

#### Run: Option 1 - locally
Note: try to run on a small number of CPUs with no GPUs, because system in non-equilibrated.

In [80]:
%%bash
cd GMX_run
gmx mdrun -ntomp 10 -deffnm 1_minim 2> 1_minim.sterr  # if you want to run locally, nt is the number of threads
echo "Outputting gromacs error log"
cat 1_minim.sterr

Outputting gromacs error log
gmx: Relink `/opt/miniconda3/envs/moldyn/bin/../lib/././libgfortran.so.4' with `/lib/x86_64-linux-gnu/librt.so.1' for IFUNC symbol `clock_gettime'
                      :-) GROMACS - gmx mdrun, 2018.4 (-:

                            GROMACS is written by:
     Emile Apol      Rossen Apostolov      Paul Bauer     Herman J.C. Berendsen
    Par Bjelkmar    Aldert van Buuren   Rudi van Drunen     Anton Feenstra  
  Gerrit Groenhof    Aleksei Iupinov   Christoph Junghans   Anca Hamuraru   
 Vincent Hindriksen Dimitrios Karkoulis    Peter Kasson        Jiri Kraus    
  Carsten Kutzner      Per Larsson      Justin A. Lemkul    Viveca Lindahl  
  Magnus Lundborg   Pieter Meulenhoff    Erik Marklund      Teemu Murtola   
    Szilard Pall       Sander Pronk      Roland Schulz     Alexey Shvetsov  
   Michael Shirts     Alfons Sijbers     Peter Tieleman    Teemu Virolainen 
 Christian Wennberg    Maarten Wolf   
                           and the project leaders:
   

### Troubleshooting exploding trajectories !!! (if needed)
------

In [81]:
#First steps often require troubleshooting and looking in vmd at the structure.
#To do it you'll need to change MD_Protocols/1_minim.mdp to decrease number of steps to 20 or 100, and set output for every step.
#Let's generate useful commands that can be used on your local machine.
# See if there are any real, problems or clashed, LINCS warnings are usually sensitive to that.
#If there are still unexplainable problems with LINCS, try to change constrains = None in 1_minim.mdp file, usually this helps.
dir=!pwd
newton_user=!whoami
!cp VMD_scripts/min_troubleshoot.tcl GMX_run/
!cp GMX_system/init_solv_ions.pdb GMX_run
with open('GMX_run/1_minim.sterr', "r") as file:
    counter=0
    line='1'
    atoms=[]
    while ((counter<1000) and line):
        line=file.readline()
        m=re.match("\s+(\d+)\s+\d+\s+\d+\.\d+\s+\d+\.\d+\s+\d+\.\d+\s+\d+\.\d+",line)
        if(m):
            atoms.append(m.groups()[0])
            counter+=1
        else:
            next
    atomstr=' '.join(atoms)
if(len(atoms)>0):
    !echo 'mol representation VDW' >> GMX_run/min_troubleshoot.tcl
    !echo "mol selection index $atomstr" >> GMX_run/min_troubleshoot.tcl
    !echo "mol addrep top" >> GMX_run/min_troubleshoot.tcl
#!echo "$atomstr"
print("rm -r GMX_run")
print("rsync -avz --exclude='step*' --exclude='\#*' %s@newton.bioeng.ru:%s/GMX_run ."%(newton_user[0],dir[0]))
print("vmd -e GMX_run/min_troubleshoot.tcl\n")

rm -r GMX_run
rsync -avz --exclude='step*' --exclude='\#*' alexsha@newton.bioeng.ru:/home/_shared/_projects/nucl_gmx/1kx5_sym/GMX_run .
vmd -e GMX_run/min_troubleshoot.tcl



------------


#### Run: Option 2 - sending and simulating on Lomonosov-2
Note: Usually it is easier just to run all the cells for local simulations below and wait some time, than to send to supercomputer since it requires manual work and monitoring. But examples of doing it on supercomputer are provided for the next two step for reference.
They are commented out (set to RAW content), so that this note book could be run entirely as a single piece of code locally.

In [ ]:
deffnm='1_minim'

#### Analyze what we get

### 2_Equilibration 100 ps with postional restraints of 500 kJ/mol/A2 , and with 0.5 fs time step

In [82]:
%%bash  
#--out out --err err
gmx grompp -f MDProtocols/2_equil.mdp -c GMX_run/1_minim.gro -p GMX_system/topol.top \
-o GMX_run/2_equil.tpr -v  -r GMX_system/init_solv_ions.pdb -n GMX_system/index.ndx -po GMX_run/mdout_2_equil.mdp


processing topology...
turning all bonds into constraints...
turning all bonds into constraints...
turning all bonds into constraints...
turning all bonds into constraints...
turning all bonds into constraints...
turning all bonds into constraints...
turning all bonds into constraints...
turning all bonds into constraints...
turning all bonds into constraints...
turning all bonds into constraints...
turning all bonds into constraints...
turning all bonds into constraints...
turning all bonds into constraints...
Determining Verlet buffer for a tolerance of 0.005 kJ/mol/ps at 300 K
Calculated rlist for 1x1 atom pair-list as 0.801 nm, buffer size 0.001 nm
Set rlist, assuming 4x4 atom pair-list, to 0.800 nm, buffer size 0.000 nm
Note that mdrun will redetermine rlist based on the actual pair-list setup
Calculating fourier grid dimensions for X Y Z
Using a fourier grid of 160x160x160, spacing 0.116 0.116 0.116
This run will generate roughly 74 Mb of data


gmx: Relink `/opt/miniconda3/envs/moldyn/bin/../lib/././libgfortran.so.4' with `/lib/x86_64-linux-gnu/librt.so.1' for IFUNC symbol `clock_gettime'
                      :-) GROMACS - gmx grompp, 2018.4 (-:

                            GROMACS is written by:
     Emile Apol      Rossen Apostolov      Paul Bauer     Herman J.C. Berendsen
    Par Bjelkmar    Aldert van Buuren   Rudi van Drunen     Anton Feenstra  
  Gerrit Groenhof    Aleksei Iupinov   Christoph Junghans   Anca Hamuraru   
 Vincent Hindriksen Dimitrios Karkoulis    Peter Kasson        Jiri Kraus    
  Carsten Kutzner      Per Larsson      Justin A. Lemkul    Viveca Lindahl  
  Magnus Lundborg   Pieter Meulenhoff    Erik Marklund      Teemu Murtola   
    Szilard Pall       Sander Pronk      Roland Schulz     Alexey Shvetsov  
   Michael Shirts     Alfons Sijbers     Peter Tieleman    Teemu Virolainen 
 Christian Wennberg    Maarten Wolf   
                           and the project leaders:
        Mark Abraham, Berk Hess

#### Run: Option 1 - locally

In [ ]:
%%bash
cd GMX_run
gmx mdrun -ntomp 10 -deffnm 2_equil # if you want to run locally

#### Run: Option 2 - sending and simulating on Lomonosov-2
Note: Usually it is easier just to run all the cells for local simulations below and wait some time, than to send to supercomputer since it requires manual work and monitoring. But examples of doing it on supercomputer are provided for this step for reference. The same can be applied to all further step if needed - just copy the code.
The code here is commented out (cell type set to RAW content), so that this notebook could be run entirely as a single piece of code locally.

In [ ]:
deffnm='2_equil'

#### Analyze what we get

### 3_Equilibration 200 ps with postional restraints of 50 kJ/mol/A2

In [ ]:
%%bash  
#--out out --err err
gmx grompp -f MDProtocols/3_equil.mdp -c GMX_run/2_equil.gro -p GMX_system/topol.top \
-o GMX_run/3_equil.tpr -v  -r GMX_system/init_solv_ions.pdb -n GMX_system/index.ndx -po GMX_run/mdout_3_equil.mdp


#### Run: Option 1 - locally

In [ ]:
%%bash
cd GMX_run
gmx mdrun -ntomp 10 -deffnm 3_equil # if you want to run locally

#### Run: Option 2 - sending and simulating on Lomonosov-2

In [ ]:
deffnm='3_equil'

### 4_Equilibration 200 ps with postional restraints of 5 kJ/mol/A2

In [ ]:
%%bash  
#--out out --err err
gmx grompp -f MDProtocols/4_equil.mdp -c GMX_run/3_equil.gro -p GMX_system/topol.top \
-o GMX_run/4_equil.tpr -v  -r GMX_system/init_solv_ions.pdb -n GMX_system/index.ndx -po GMX_run/mdout_4_equil.mdp


#### Run: Option 1 - locally

In [ ]:
%%bash
cd GMX_run
gmx mdrun -ntomp 10 -deffnm 4_equil # if you want to run locally

#### Run: Option 2 - sending and simulating on Lomonosov-2

In [ ]:
deffnm='4_equil'

### 5_Equilibration 200 ps with postional restraints of 0.5 kJ/mol/A2

In [ ]:
%%bash  
#--out out --err err
gmx grompp -f MDProtocols/5_equil.mdp -c GMX_run/4_equil.gro -p GMX_system/topol.top \
-o GMX_run/5_equil.tpr -v  -r GMX_system/init_solv_ions.pdb -n GMX_system/index.ndx -po GMX_run/mdout_5_equil.mdp


#### Run: Option 1 - locally

In [ ]:
%%bash
cd GMX_run
gmx mdrun -ntomp 10 -deffnm 5_equil # if you want to run locally

#### Run: Option 2 - sending and simulating on Lomonosov-2

In [ ]:
deffnm='5_equil'

### 6_Equilibration 200 ps unrestrained

In [ ]:
%%bash  
#--out out --err err
gmx grompp -f MDProtocols/6_equil.mdp -c GMX_run/5_equil.gro -p GMX_system/topol.top \
-o GMX_run/6_equil.tpr -v  -r GMX_system/init_solv_ions.pdb -n GMX_system/index.ndx -po GMX_run/mdout_6_equil.mdp


#### Run: Option 1 - locally

In [ ]:
%%bash
cd GMX_run
gmx mdrun -ntomp 10 -deffnm 6_equil # if you want to run locally

#### Run: Option 2 - sending and simulating on Lomonosov-2

In [ ]:
deffnm='6_equil'

# Saving final equilibrated results

As a result we have equlibrated system ready for production simulations.
We copy it to GMX_system.

In [ ]:
%%bash
cp GMX_run/6_equil.gro GMX_system/ #coordinates
cp GMX_run/6_equil.cpt GMX_system/ #This has also velocities





Optionally we can save all equilibrated files, if want to copy them somewhere and user later,
here we place into results folder the set of equilibrated files of step 6.


In [ ]:
%%bash
gmx trjconv -f GMX_run/6_equil.gro -s GMX_run/6_equil.tpr -o GMX_run/6_equil.pdb << !
0
!

In [ ]:
%%bash
mkdir -p results
cp GMX_run/6_equil* results/
cp GMX_system/sys_ref.pdb results/